In [94]:
import pandas as pd
from pandas import ExcelWriter
import re

df_parse = pd.read_excel('pars_personal_information.xlsx')
df_kristina = pd.read_excel('2021_12_kristina.xlsx')

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:211: UserWarning: Cell J1898 is marked as a date but the serial value 9139164845 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:211: UserWarning: Cell K1898 is marked as a date but the serial value 89151586026 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


### Смотрим и сохраняем проблемные значения в частности дубликаты

In [95]:
# создаем df где пропущены значения "номер билета", сохраняем и кикаем
df_without_numbers = df_kristina[df_kristina['номер билета'].isnull()]
with ExcelWriter('uncorrect_id.xlsx') as writer:
    df_without_numbers.to_excel(writer, sheet_name='Sheet1')
    
# создаем df где дубли членских билето    
df_duplicated_id = df_kristina[df_kristina['номер билета'].duplicated(keep=False)]
with ExcelWriter('duplicated_id.xlsx') as writer:
    df_duplicated_id.to_excel(writer, sheet_name='Sheet1')

In [96]:
# Две функции для очистки читательских от лишних знаков
def clean_spaces(row):
    try:
        pattern = r'[1-9]\d+[-]*'      
        return ''.join(re.findall(pattern, row))
    except:
        return row
    
def check_n(row):
    return row.replace('\n', '')

# print(check_n('324-234\n'))    
# print(clean_spaces('342324 -  324'))
# print(clean_spaces('000342324-324'))
# print(clean_spaces('34232safd4-324'))

In [97]:
df_kristina = df_kristina.dropna(subset=['номер билета'])  # Удаляем нули в номер билета
df_kristina = df_kristina.drop_duplicates(subset=['номер билета'], keep=False)  # Кикаем дубликаты
df_kristina['member_id_card'] = df_kristina['номер билета'].apply(clean_spaces)  # Чистим с помощью нашей функции
df_parse['member_id_card'] = df_parse['member_id_card'].apply(check_n)  # Чистим с помощью нашей функции
df_merge = df_kristina.merge(df_parse, on='member_id_card', how='outer')  # Склеиваем два df

In [98]:
# Попытка объединить фреймы по именам... не увенчалася успехом
# df_test = pd.read_excel('2021_12_kristina.xlsx')
# df_duplicated_id = df_test[df_test['номер билета'].duplicated(keep=False)]
# df_duplicated_id.drop(columns='номер билета', inplace=True)
# df_duplicated_id['fullname'] = df_duplicated_id['ФИО'].copy()
# df_test = df_merge.copy()
# df_test = pd.concat([df_test, df_duplicated_id], ignore_index=True, join='inner')
# df_merge = df_merge.merge(df_duplicated_id, on = 'fullname', how='left', left_indexbool=True)

In [99]:
df_uncorrect_names_or_numers = df_merge[df_merge['ФИО'] != df_merge['fullname']]
df_uncorrect_numbers_again = df_merge[df_merge['fullname'].isnull()]
with ExcelWriter('ncorrect_numbers_again.xlsx') as writer:
    df_uncorrect_numbers_again.to_excel(writer, sheet_name='Sheet1')

In [100]:
# Убираем нули в fullname, создаем файл с дублями читательских
df_merge = df_merge[df_merge['fullname'].notnull()]
df_merge = df_merge.reset_index(drop=True)
df_dupl = df_merge[df_merge['member_id_card'].duplicated(keep=False)]
with ExcelWriter('duplicated_id_again.xlsx') as writer:
    df_dupl.to_excel(writer, sheet_name='Sheet1')

In [101]:
# В ручную кикнули дубли
df_merge.drop(labels=[107, 864, 3429],axis = 0, inplace=True)

# Удаляем дурацкие колонки "Привет Эксель"
for i in range(28,50):
    df_merge = df_merge.drop(columns=[f'Unnamed: {i}'])

In [102]:
# Функий сверки имен
def check_names(row):
    try:
        if row['ФИО'].split() == row['fullname'].split():
            return row['fullname']
        else:
            return row['ФИО']  
    except:
        return row['fullname']

In [103]:
# Удаляем ошибочные именна
df_merge['ФИО'] = df_merge.apply(check_names, axis=1)
df_test = df_merge[df_merge['ФИО'] != df_merge['fullname']]
with ExcelWriter('mistakes_names.xlsx') as writer:
    df_test.to_excel(writer, sheet_name='Sheet1')

In [104]:
# Другая функция, для проверки уже имен с ошибками... регистр не проверяется.
def correct_name(row):
    name_kris = row['ФИО']
    name_pars = row['fullname']
    if row['ФИО'] == row['fullname']:
        return name_pars
    for value in row['fullname'].split():
        if value in row['ФИО'].split():
            correct_name = f'{name_kris}\n({name_pars})'
            return correct_name
    correct_name = f'{name_pars} \n (возможна ошибка)'
    return correct_name
    
# columns_name = ['ФИО', 'fullname']  # Проверка работоспособности функции
# values = [['Иванов Иван Иванович', 'Иванов Иван Иванович'],
#           ['Иванов Иван Иванович', 'Иуанов Иван Иванович'],
#           ['Иванов Иван Иванович', 'просто член']]
# test_func_df = pd.DataFrame(data=values, columns=columns_name)
# test_func_df['fullname'] = test_func_df.apply(correct_name, axis=1)
# test_func_df

In [105]:
# Создаем столбец с правильным именем
df_merge['correct_name'] = df_merge.apply(correct_name, axis=1)

In [106]:
# Импорт полученного df на всякий случай перед глобальным удалением данных
df_merge.to_csv('important_moment.csv')

### Удаление строк

In [107]:
# Странные ребята с непонятными данными вступления
df_test = df_merge[df_merge['member_year'].isnull()]
df_test = df_test[df_test['Год вступления'].notnull()]
with ExcelWriter('data_stranges.xlsx') as writer:
    df_test.to_excel(writer, sheet_name='Sheet1')

### Чистка телефонов и email

In [108]:
def phones_update(uncorrect_phone):
    uncorrect_phone = str(uncorrect_phone)
    pattern = r'\d'
    new_phone = re.findall(pattern, uncorrect_phone)
    if len(new_phone) == 0:
        return None
    new_phone = ''.join(new_phone)
    
    if len(new_phone) == 11 and new_phone[0] == '8':
        new_phone = re.sub('8', '+7', new_phone, count=1)
    if len(new_phone) == 11 and new_phone[0] == '7':
        new_phone = re.sub('7', '+7', new_phone, count=1)
    if len(new_phone) == 10 and new_phone[0] == '9':
        new_phone = '+7' + new_phone
    if len(new_phone) == 10 and new_phone[0] == '4' and new_phone[1] == '9':
        new_phone = '+7' + new_phone
    return new_phone

In [109]:
df_merge['phone_one'] = df_merge['Телефон мобильный'].apply(phones_update)
df_merge['phone_two'] = df_merge['mphone'].apply(phones_update)
df_merge['phone_three'] = df_merge['Телефон'].copy() # В этой колонке много телефонов "сразу по два", поэтому копируем

In [110]:
columns_to_delete = ['Телефон мобильный', 'mphone', 'lastname', 'first_name', 'ФИО',
                     'middle_name', 'country', 'member_region', 'member_id_card_status',
                     'adress', 'Unnamed: 0', 'Телефон', 'номер билета', 'Дата рождения', 
                     'Номер паспорта', 'Год вступления', 'Паспортные данные', 'fullname']

for column in columns_to_delete:
    df_merge.drop(columns=column, inplace=True)

In [111]:
df_merge = df_merge[['member_id_card', 'correct_name', 'member_status', 'page', 'member_data', 'member_year',
       'phone_one', 'phone_two', 'phone_three',    
       'email', 'e-mail', '2-й эл. адрес', 'birthday',
       'Пометка', 'БИЛЕТЫ', 'ПРИМЕЧАНИЕ о билете','№ Протокола о приеме', 'ПРИМЕЧАНИЕ о контакте', 
       'Деятельность', 'Должность, описание', 'work_place', 'type_employment', 'Регион', 
       'Оплата 2015', 'Оплата 2016', 'Оплата 2017', 'Оплата 2018', 'Оплата 2019',
       'Оплата 2020', 'Оплата 2021', 'Оплата 2022', 'Оплата 2023', 'Оплата 2024']]

In [ ]:
with ExcelWriter('tkcb.xlsx') as writer:
    df_merge.to_excel(writer, sheet_name='Sheet1')